In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def point_segment_distance(x, a, b):
    x = np.asarray(x, dtype=float)
    a = np.asarray(a, dtype=float)
    b = np.asarray(b, dtype=float)

    v = b - a
    w = x - a
    vv = np.dot(v, v)

    if vv == 0.0:
        return np.linalg.norm(x - a)

    t = np.dot(w, v) / vv
    t = np.clip(t, 0.0, 1.0)

    q = a + t * v
    return np.linalg.norm(x - q)


In [ ]:
walls = [
    (np.array([1,1]), np.array([8,1])),
    (np.array([8,1]), np.array([8,6])),
    (np.array([8,6]), np.array([3,6])),
    (np.array([3,6]), np.array([3,9])),
]


In [ ]:
start = np.array([2,2])
goal  = np.array([7,8])


In [ ]:
def goal_cost(x):
    return 0.5 * np.linalg.norm(x - goal)**2

In [ ]:
def phi_quadratic(d, R):
    if d <= R:
        return 0.5 * (R - d)**2
    return 0.0

In [ ]:
def phi_log(d, R, eps=1e-3):
    if d <= R:
        return np.log(R / (d + eps))
    return 0.0

In [ ]:
def phi_inverse(d, R, eps=1e-3, p=2):
    if d <= R:
        return 1.0 / ((d + eps)**p)
    return 0.0

In [ ]:
def wall_cost(x, phi, R, w=50):
    total = 0.0
    for a,b in walls:
        d = point_segment_distance(x, a, b)
        total += w * phi(d, R)
    return total

In [ ]:
def total_cost(x, phi, R, w=50):
    return goal_cost(x) + wall_cost(x, phi, R, w)

In [ ]:
def numerical_gradient(f, x, h=1e-5):
    grad = np.zeros_like(x)
    for i in range(len(x)):
        x1 = x.copy()
        x2 = x.copy()
        x1[i] += h
        x2[i] -= h
        grad[i] = (f(x1) - f(x2)) / (2*h)
    return grad

In [ ]:
def gradient_descent(phi, R, w=50, alpha=0.05, max_iters=500):
    x = start.copy()
    trajectory = [x.copy()]

    for _ in range(max_iters):
        f = lambda z: total_cost(z, phi, R, w)
        grad = numerical_gradient(f, x)
        x = x - alpha * grad
        trajectory.append(x.copy())

        if np.linalg.norm(x - goal) < 0.1:
            break

    return np.array(trajectory)

In [2]:
def plot_field(phi, R, w=50):
    xs = np.linspace(0,10,200)
    ys = np.linspace(0,10,200)
    X, Y = np.meshgrid(xs, ys)
    Z = np.zeros_like(X)

    for i in range(X.shape[0]):
        for j in range(X.shape[1]):
            Z[i,j] = total_cost(np.array([X[i,j], Y[i,j]]), phi, R, w)

    plt.figure(figsize=(8,6))
    plt.contourf(X, Y, Z, levels=50)
    plt.colorbar()

    # Gradient vectors
    step = 15
    for i in range(0, X.shape[0], step):
        for j in range(0, X.shape[1], step):
            point = np.array([X[i,j], Y[i,j]])
            f = lambda z: total_cost(z, phi, R, w)
            grad = numerical_gradient(f, point)
            plt.arrow(point[0], point[1], -grad[0]*0.05, -grad[1]*0.05,
                      head_width=0.1, color='white')

    # Walls
    for a,b in walls:
        plt.plot([a[0], b[0]], [a[1], b[1]], 'k', linewidth=3)

    plt.scatter(*start, c='green', s=100)
    plt.scatter(*goal, c='red', s=100)

    plt.xlim(0,10)
    plt.ylim(0,10)
    plt.title("Cost Contours + Gradient Field")
    plt.show()

In [ ]:
def plot_trajectory(traj):
    plt.figure(figsize=(8,6))

    for a,b in walls:
        plt.plot([a[0], b[0]], [a[1], b[1]], 'k', linewidth=3)

    plt.plot(traj[:,0], traj[:,1], 'b-', linewidth=2)
    plt.scatter(*start, c='green', s=100)
    plt.scatter(*goal, c='red', s=100)

    plt.xlim(0,10)
    plt.ylim(0,10)
    plt.title("Gradient Descent Trajectory")
    plt.show()

In [ ]:
R = 1.5
traj_quad = gradient_descent(phi_quadratic, R)
plot_field(phi_quadratic, R)
plot_trajectory(traj_quad)

In [ ]:
traj_log = gradient_descent(phi_log, R)
plot_field(phi_log, R)
plot_trajectory(traj_log)

In [3]:
traj_inv = gradient_descent(phi_inverse, R)
plot_field(phi_inverse, R)
plot_trajectory(traj_inv)

NameError: name 'gradient_descent' is not defined